<a href="https://colab.research.google.com/github/JuanLara18/Actuarial-DataMined-Provisiones/blob/main/notebooks/Project_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The data can be download [here](https://www.casact.org/publications-research/research/research-resources/loss-reserving-data-pulled-naic-schedule-p).

# Data Understanding

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

data = pd.read_csv("wkcomp_pos.csv")

In [2]:
# Display basic information about the dataset
print("Dataset Information:")
print("Number of Records:", data.shape[0])
print("Number of Attributes:", data.shape[1])

Dataset Information:
Number of Records: 13200
Number of Attributes: 13


In [3]:
# Display data types of attributes
print("\nData Types of Attributes:")
print(data.dtypes)


Data Types of Attributes:
GRCODE                int64
GRNAME               object
AccidentYear          int64
DevelopmentYear       int64
DevelopmentLag        int64
IncurLoss_D           int64
CumPaidLoss_D         int64
BulkLoss_D            int64
EarnedPremDIR_D       int64
EarnedPremCeded_D     int64
EarnedPremNet_D       int64
Single                int64
PostedReserve97_D     int64
dtype: object


In [4]:
# Check unique values and coding schemes for specific attributes
print("\nCoding Schemes:")
print("Number of unique values in 'GRCODE':", len(data['GRCODE'].unique()))
print("Number of unique values in 'GRNAME':", len(data['GRNAME'].unique()))
print("Number of unique values in 'AccidentYear':", len(data['AccidentYear'].unique()))
print("Number of unique values in 'DevelopmentYear':", len(data['DevelopmentYear'].unique()))


Coding Schemes:
Number of unique values in 'GRCODE': 132
Number of unique values in 'GRNAME': 132
Number of unique values in 'AccidentYear': 10
Number of unique values in 'DevelopmentYear': 19


# Data Preparation and Statistics Related

In [5]:
# Extract de information in DataFrames
Important_Variables = data.columns.delete([0,1,2,3,4,-1,-2,-3,-4,-5])
DataComplete = {column : [] for column in Important_Variables}
size = data.shape[0]

for Variable in Important_Variables:
  count = 0
  while(size - 100*count > 1):
    base_Variable = np.zeros((10,10))
    for i in range(10):
      for j in range(10):
        base_Variable[i][j] = data[Variable][i*10+j + 10*count]
    count = count + 1
    DataComplete[Variable].append(pd.DataFrame(base_Variable))

In [6]:
# Construct the Triangles
def IncompleteDataFrame(x):
  matrix = x.values
  for i in range(9,0,-1):
    for j in range(10-i,10,1):
        matrix[i][j] = 0
  return pd.DataFrame(matrix)

Triangles = {}
for variable, array in DataComplete.items():
  Triangles[variable] = [IncompleteDataFrame(x) for x in array]

In [ ]:
# Find the basics statistics component by component for each variable
Means = {}
for variable, array in Triangles.items():
  Means[variable] = pd.concat(array).groupby(level=0).mean()

Var = {}
for variable, array in Triangles.items():
  Var[variable] = pd.concat(array).groupby(level=0).var()

Min = {}
for variable, array in Triangles.items():
  Min[variable] = pd.concat(array).groupby(level=0).min()

Max = {}
for variable, array in Triangles.items():
  Max[variable] = pd.concat(array).groupby(level=0).max()

Median = {}
for variable, array in Triangles.items():
  Median[variable] = pd.concat(array).groupby(level=0).median()

In [ ]:
# Define the function to plot the series
def DrawGraph(x, name):
  matrix = x.values
  plt.figure(figsize=(9, 7))
  plt.plot()

  for i in range(10):
    val = matrix[i]
    for j in range(i):
      val = val[:-1]
    rang = range(1988,1988+len(val))
    plt.plot(rang, val, marker='o', linestyle='-', label=1988+i)

  plt.xlabel('Years')
  plt.ylabel('Value')
  plt.title(name)

  plt.savefig(name, dpi=300, bbox_inches='tight')

  plt.legend()

In [ ]:
# Export the graphs of the basis statistics
for var in Important_Variables:
  DrawGraph(Means[var], 'Mean: ' + var)

for var in Important_Variables:
  DrawGraph(Var[var], 'Variance: ' + var)

for var in Important_Variables:
  DrawGraph(Median[var], 'Median: ' + var)

In [ ]:
# PCA
from sklearn.decomposition import PCA

def dataframe_to_pca(dataframes, n_components):

  X = np.array([])
  for i in range(len(dataframes)):
    flattened_vector = dataframes[i].values.flatten()
    X = np.append(X, flattened_vector)

  X = X.reshape(len(dataframes), -1)

  pca = PCA(n_components=n_components)
  pca_result = pca.fit_transform(X)

  return pca_result

In [ ]:
# Outliers

In [ ]:
# Distributions / Hist

# Models

## Chain Ladder determinists

In [21]:
def chain_ladder_deterministic(data):
    development_factors = []  # Lista de factores de desarrollo
    cumulative_factors = []    # Lista de factores acumulados

    # Calcular los factores de desarrollo
    for i in range(1, len(data)):
        development_factor = sum(data[i]) / sum(data[i - 1])
        development_factors.append(development_factor)

    # Calcular los factores acumulados
    cumulative_factor = 1
    for factor in development_factors:
        cumulative_factor *= factor
        cumulative_factors.append(cumulative_factor)

    # Proyectar reclamaciones futuras
    ultimate_claim = sum(data[0]) * cumulative_factors[-1]
    projected_claims = [ultimate_claim * factor for factor in cumulative_factors]

    # Completar el triángulo de datos con las proyecciones
    full_data = data.copy()
    for i in range(len(data), len(data) + len(projected_claims)):
        full_data.append([projected_claims[i - len(data)]])

    return full_data

# Ejemplo de uso
data = Triangles['IncurLoss_D'][0]

full_data = chain_ladder_deterministic(data)

print(full_data)


          0         1         2         3         4         5         6  \
0  367404.0  362988.0  347288.0  330648.0  354690.0  350092.0  346808.0   
1  336928.0  316483.0  278496.0  303033.0  299496.0  295061.0  299251.0   
2  289198.0  311381.0  277980.0  277732.0  276563.0  278067.0  276704.0   
3  297174.0  277209.0  269739.0  272666.0  271318.0  267578.0  269592.0   
4  181796.0  205079.0  199106.0  187833.0  185663.0  184940.0       0.0   
5  114807.0  114774.0  101460.0   98430.0   96930.0       0.0       0.0   
6  107934.0  107569.0   97730.0   96185.0       0.0       0.0       0.0   
7  100686.0   94456.0   92314.0       0.0       0.0       0.0       0.0   
8   53381.0   51205.0       0.0       0.0       0.0       0.0       0.0   
9    6725.0       0.0       0.0       0.0       0.0       0.0       0.0   

          7         8         9  
0  349124.0  348157.0  347762.0  
1  297492.0  300620.0       0.0  
2  281101.0       0.0       0.0  
3       0.0       0.0       0.0  
4   

<ipython-input-21-7e29dce78586>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data.append([projected_claims[i - len(data)]])


In [14]:
Triangles.keys()

dict_keys(['IncurLoss_D', 'CumPaidLoss_D', 'BulkLoss_D'])

In [15]:
Triangles['IncurLoss_D'][0]

,0,1,2,3,4,5,6,7,8,9
0,367404.0,362988.0,347288.0,330648.0,354690.0,350092.0,346808.0,349124.0,348157.0,347762.0
1,336928.0,316483.0,278496.0,303033.0,299496.0,295061.0,299251.0,297492.0,300620.0,0.0
2,289198.0,311381.0,277980.0,277732.0,276563.0,278067.0,276704.0,281101.0,0.0,0.0
3,297174.0,277209.0,269739.0,272666.0,271318.0,267578.0,269592.0,0.0,0.0,0.0
4,181796.0,205079.0,199106.0,187833.0,185663.0,184940.0,0.0,0.0,0.0,0.0
5,114807.0,114774.0,101460.0,98430.0,96930.0,0.0,0.0,0.0,0.0,0.0
6,107934.0,107569.0,97730.0,96185.0,0.0,0.0,0.0,0.0,0.0,0.0
7,100686.0,94456.0,92314.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,53381.0,51205.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,6725.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
